<a href="https://colab.research.google.com/github/MaryaMalyshkina/prolog/blob/main/prolog_1500_300.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision Pillow -q

In [2]:
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.3 MB/s eta 0:00:00


In [3]:
import torch
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib.figure import Figure
from pathlib import Path
import random
import shutil

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from ultralytics import YOLO

# Load a pre-trained YOLO11 instance segmentation model
model = YOLO("yolo11n-seg.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
# Проверка доступности GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [7]:
results = model.train(
    data="/content/drive/MyDrive/svik54.v3-prolog_1500.yolov11/data.yaml",
    epochs=300,

    imgsz=640,
    batch=16,
    workers=2,

    # Гиперпараметры
    lr0=0.01,                 # начальный learning rate
    lrf=0.001,                 # конечный learning rate
    momentum=0.937,           # момент SGD
    weight_decay=0.0005,      # весовая регуляризация
    warmup_epochs=3,          # прогрев
    warmup_momentum=0.8,      # момент при прогреве
    warmup_bias_lr=0.1,       # lr для bias при прогреве


    patience=20,              # ранняя остановка если нет улучшений

    # Оптимизация
    optimizer="SGD",          # можно "AdamW" для маленького датасета
    cos_lr=True,              # косинусный scheduler
    close_mosaic=10,          # отключить mosaic за 10 эпох до конца

    # Валидация
    val=True,                 # валидация после каждой эпохи
    save_period=10,           # сохранять чекпоинты каждые 10 эпох

    # Другое
    device="cuda" if torch.cuda.is_available() else "cpu",
    seed=42,                  # для воспроизводимости
    deterministic=True,       # детерминированные алгоритмы
    single_cls=False,         # мультиклассовая классификация
    plots=True,               # строить графики
    verbose=True,             # подробный вывод
)

model.save("prolog_1500_300_model.pt")

model.save("/content/drive/MyDrive/prolog_1500_300_model.pt")
print("Модель сохранена в Google Drive: MyDrive/prolog_1500_300_model.pt")

Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/svik54.v3-prolog_1500.yolov11/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.001, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=20, p

In [8]:
# Валидация модели
metrics = model.val(data="/content/drive/MyDrive/svik54.v3-prolog_1500.yolov11/data.yaml")

print("Метрики детекции:")
print(f"  mAP50: {metrics.box.map50:.4f}")
print(f"  mAP50-95: {metrics.box.map:.4f}")
print(f"  Precision: {np.mean(metrics.box.p):.4f}")

print(f"  Recall: {np.mean(metrics.box.r):.4f}")

print("\nМетрики сегментации:")
print(f"  Mask mAP50: {metrics.seg.map50:.4f}")
print(f"  Mask mAP50-95: {metrics.seg.map:.4f}")

Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-seg summary (fused): 113 layers, 2,835,153 parameters, 0 gradients, 9.6 GFLOPs
val: Fast image access ✅ (ping: 0.5±0.2 ms, read: 24.5±3.8 MB/s, size: 41.8 KB)
val: Scanning /content/drive/MyDrive/svik54.v3-prolog_1500.yolov11/valid/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 300/300 429.7Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 19/19 2.9it/s 6.5s
                   all        300        372      0.971      0.943      0.979      0.873      0.935      0.974      0.975      0.854
                 block         21         26          1      0.909      0.995      0.953      0.993          1      0.995      0.956
                 crack        103        116      0.969       0.94      0.963       0.82      0.891       0.94       0.95      0.744
           

In [9]:
source_dir = "/content/runs/segment"
destination_base = "/content/drive/MyDrive/prolog_1500_300.v1i.yolov11/runs"

# Копируем всю папку
shutil.copytree(source_dir, destination_base)

'/content/drive/MyDrive/prolog_1500_300.v1i.yolov11/runs'

In [10]:
test_folder = "/content/drive/MyDrive/svik54.v3-prolog_1500.yolov11/test/images"
output_dir = "/content/drive/MyDrive/svik54.v3-prolog_1500.yolov11/test_predictions"

os.makedirs(output_dir, exist_ok=True)

results = model.predict(
    source=test_folder,
    conf=0.5,
    save=True,
    save_txt=True,
    save_conf=True,
    save_crop=False,
    project=output_dir,
    name="test_run",
    exist_ok=True,
    verbose=True
)

print(f"\n Все предсказания сохранены в: {output_dir}/test_run")


image 1/146 /content/drive/MyDrive/svik54.v3-prolog_1500.yolov11/test/images/T-00_56_48-C-Side-P-84-4-A-319-9-L-0_png.rf.a40c896d4da852b0198aeac0d342e9a6.jpg: 640x640 1 joint, 11.0ms
image 2/146 /content/drive/MyDrive/svik54.v3-prolog_1500.yolov11/test/images/T-00_58_23-C-Side-P-32-7-A-160-0-L-0_png.rf.5bb45a5922d5410a5ddc511dcd1c14f4.jpg: 640x640 1 joint, 322.0ms
image 3/146 /content/drive/MyDrive/svik54.v3-prolog_1500.yolov11/test/images/T-01-16-33-C-Side-P-5347-5-A-40-1-L-0_png.rf.c61fc285f82acb6b59da2f6fc104f0f6.jpg: 640x640 (no detections), 15.0ms
image 4/146 /content/drive/MyDrive/svik54.v3-prolog_1500.yolov11/test/images/T-01-20-37-C-Side-P-5436-9-A-90-6-L-0_png.rf.d0b9da3675dff90829f34f689612528a.jpg: 640x640 1 crack, 1 joint, 25.2ms
image 5/146 /content/drive/MyDrive/svik54.v3-prolog_1500.yolov11/test/images/T-01-23-31-C-Side-P-4676-3-A-93-6-L-0_png.rf.00f8218f743232950473d3bff56fb3d2.jpg: 640x640 2 cracks, 2 joints, 15.4ms
image 6/146 /content/drive/MyDrive/svik54.v3-prolog_